In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cuda


In [14]:
df = pd.read_csv("/kaggle/input/telaaviv/dxb_telaviv_8965d2_cleaned.csv")
df.head()

,time,icao24,lat,lon,velocity,heading,vertrate,baroaltitude,hour
0,1734701441,8965d2,25.275925,55.248261,137.182066,245.164832,11.37920,1066.8,1734699600
1,1734701442,8965d2,25.275925,55.248261,137.182066,245.164832,11.37920,1066.8,1734699600
2,1734701443,8965d2,25.275925,55.248261,137.182066,245.164832,11.05408,1066.8,1734699600
3,1734701444,8965d2,25.275925,55.248261,137.182066,245.164832,11.05408,1066.8,1734699600
4,1734701445,8965d2,25.275925,55.248261,137.182066,245.164832,11.05408,1066.8,1734699600


In [ ]:
features = ['lat', 'lon', 'baroaltitude']
seq_len = 10
target_idx = 0  
data = df[features].values

In [16]:
def create_sequences(data, target_idx, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len][target_idx])
    return np.array(X), np.array(y)

X_seq, y_seq = create_sequences(data, target_idx=target_idx, seq_len=seq_len)


In [17]:
X_tensor = torch.tensor(X_seq, dtype=torch.float32)
y_tensor = torch.tensor(y_seq, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [18]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Take last timestep output
        return self.fc(out)


In [19]:
model = LSTMRegressor(input_dim=X_seq.shape[2], hidden_dim=64).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [20]:
epochs = 20
model.train()

for epoch in range(epochs):
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        preds = model(xb).squeeze()
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


Epoch 1/20, Loss: 202380.6356
Epoch 2/20, Loss: 6520.2771
Epoch 3/20, Loss: 10688.4228
Epoch 4/20, Loss: 4311.3874
Epoch 5/20, Loss: 3192.2695
Epoch 6/20, Loss: 2383.1075
Epoch 7/20, Loss: 2060.4732
Epoch 8/20, Loss: 2141.4392
Epoch 9/20, Loss: 2193.8017
Epoch 10/20, Loss: 2207.8576
Epoch 11/20, Loss: 1924.6780
Epoch 12/20, Loss: 2551.2684
Epoch 13/20, Loss: 1712.1384
Epoch 14/20, Loss: 1828.8994
Epoch 15/20, Loss: 1624.6852
Epoch 16/20, Loss: 1475.0684
Epoch 17/20, Loss: 1338.0612
Epoch 18/20, Loss: 1568.6993
Epoch 19/20, Loss: 1328.4019
Epoch 20/20, Loss: 1497.5919


In [23]:
from torch.utils.data import DataLoader

model.eval()
preds = []

predict_loader = DataLoader(TensorDataset(torch.tensor(X_seq, dtype=torch.float32)),
                            batch_size=64, shuffle=False)

with torch.no_grad():
    for xb in predict_loader:
        xb = xb[0].to(device)
        output = model(xb).cpu().numpy().squeeze()
        preds.extend(output)

preds = np.array(preds)
print("Sample predictions:", preds[:10])
print("Sample ground truths:", y_seq[:10])


Sample predictions: [25.339735 25.339735 25.339735 25.339735 25.339735 25.331356 25.331755
 25.331348 25.33063  25.330732]
Sample ground truths: [25.27592468 25.27592468 25.27592468 25.27592468 25.26962539 25.26902021
 25.26887512 25.26836848 25.26836848 25.26836848]
